In [94]:
# 환경 차이 문제로 코드 작성 안 함

# 10장 - 여러 데이터 소스를 통합 데이터셋으로 합치기

In [79]:
# 원본 데이터를 읽어들이고 전처리하는 루틴 제작
# 1단계인 데이터 로딩에 집중
# 목표는 원본 CT 스캔 데이터와 데이터에 달아놓은 애노테이션 목록으로 훈련 샘플 만드는 것

## 1절 - 원본 CT 데이터 파일

In [80]:
# CT 데이터는 메타데이터 헤더 정보가 포함된 .mhd 파일과 3차원 배열을 만들 원본 데이터 바이트를 포함하는 .raw 파일로 종류는 총 두 가지
# 각 파일 이름은 '시리즈 UID'라 불리는 CT 스캔 단일 식별자로 시작
# Ex) 시리즈 UID 1.2.3의 경우 1.2.3.mhd, 1.2.3.raw의 두 가지 파일

In [81]:
# Ct 클래스는 두 파일 읽어서 3차원 배열 만들고 환자 좌표계를 배열에서 필요로 하는 인덱스, 행, 열 좌표(I, R, C)로 바꿔주는 변환 행렬 만듫

In [82]:
# LUNA에서 제공하는 애노테이션 데이터에는 각 결절의 좌표 목록, 악성 여부, 해당 CT 스캔의 시리즈 UID 포함됨
# 결절 좌표가 좌표계 변환 정보를 거치면 결절의 중심에 해당하는 복셀의 인덱스, 행, 열 정보 생김

In [83]:
# (I, R, C) 좌표 사용하면 CT 데이터의 작은 3차원 부분 단면을 얻어 모델에 대한 입력으로 사용 가능
# 이 3차원 배열과 훈련 샘플 튜플의 나머지를 구성해야 함
# 튜플에는 샘플 배열, 결절의 상태 플래그, 시리즈 UID, 결절 후보군의 CT 리스트 중 몇 번째 인덱스인지 등 포함
# Dataset 서브클래스를 통해 얻고자 하는 것이 위의 튜플이며 원본 데이터를 표준 구조의 파이토치 텐서로 변환하는 과정의 마지막 부분

In [84]:
# 데이터를 제한하거나 잘라 모델에 노이즈 끼지 않게 하는 것은 매우 중요
# 과하게 걸러 중요 시그널 날리지 않게 해야 함
# 특히, 정규화 이후 데이터의 범위가 적절한 지 확인
# 데이터에서 이상값(outlier) 제거하는 것은 데이터에 극단적 이상값 많을 때 유용

## 2절 - LUNA 애노테이션 데이터 파싱

In [85]:
# 원본 데이터를 다루는 방법을 알아야 하며 데이터가 로딩된 후 어떻게 보일지 알면 초기 실험 구조를 이해하는데 도움 됨
# LUNA에서 제공하는 CSV 파일을 먼저 파싱하여 각 CT 스캔 중 관심 있는 부분을 파악 필요
# 좌표 정보, 해당 좌표 지점이 결절인지 여부, CT 스캔에 대한 고유 식별자 얻을 것으로 기대

In [86]:
# candidates.csv 파일에는 조직 덩어리가 결절일 가능성이 있는지와 종양의 악성 여부와 그 외 정보 존재
# 이 데이터를 전체 후보 리스트 만드는 데 사용해 추후에 훈련 데이터셋과 검증 데이터셋으로 나눌 것

In [87]:
# annotations.csv 파일에는 결절로 플래그 된 후보들에 대한 정보 포함
# 이 정보를 결절 크기의 분포로 가정하여 훈련 데이터와 검증 데이터 만드는데 유용하게 사용
# 결절 크기 정보 없다면 검증셋이 큰 결절 샘플만 가져 모델의 성능이 떨어지는 것처럼 보일 수 있음

In [88]:
# 모든 표준 지도 학습 작업은 데이터를 훈련셋과 검증셋으로 나눔
# 두 데이터셋 중 하나가 실제 사용 사례와 의미적으로 다르면 모델은 우리의 생각과 다르게 동작할 수 있음
# 훈련이나 통계를 위해 수집했던 것들을 기준으로 실제 제품으로 적용할 때 예측과 다른 모습을 보일 것임

In [89]:
# 크기 순으로 정렬한 후 매 N 번째에 대해 검증셋에 넣어 분포를 반영한 검증셋 구성

In [90]:
# annotations.csv에서 제공하는 위치 정보는 candidates.csv와 정확하게 일치하지 않음
# 각 파일에서 일치하는 좌표를 잘라내면 결절의 중심부를 나타내지만 완벽하게 일치하지 않음
# 이처럼 데이터가 불일치한 경우면 다룰 가치 없는 것으로 판단하고 무시
# 나중에 서로 다른 출처로부터 데이터를 가져와 합칠 때 필요한 전형적인 작업

In [91]:
# 원본 데이터 파일을 합치는 getCandidateInfoList 함수 제작
# 각 결절 정보를 담아둘 네임드 튜플을 파일 상단에 두고 사용

In [92]:
# 위 튜플은 우리가 필요로 하는 CT 데이터가 빠져 있으니 훈련 샘플이 아님
# 전문가의 애노테이션 데이터에 대해 나름 깔끔하게 다듬어진 통합 인터페이스를 나타냄
# 모델 훈련 작업과 지저분한 데이터를 처리하는 작업을 분리하는 것을 하지 않으면 훈련 루프 더러워짐

In [93]:
# 후보 정보 리스트는 (분류를 위한 모델 훈련에 사용할) 결절의 상태와 (훈련에 사용할 좋은 분포를 얻기 위해 사용할) 결절의 직경(큰 결절과 작은 결절은 서로 다른 특성을 가질 수 있으므로) 그리고 (올바른 CT 스캔에 배치하기 위한) 순번과 (큰 CT에서 후보를 찾기 위한) 중심점 가짐
# NoduleInfoTuple 인스턴스 리스트를 만드는 함수는 인메모리 캐싱 데코레이터를 사용하고 디스크 파일 경로 얻음

In [95]:
# 일부 데이터 파일은 파싱에 시간이 걸리므로 함수 호출 결과를 메모리에 캐시
# 인메모리나 온디스크 캐싱을 적절하게 사용하여 데이터 파이프라인 속도를 올려놓으면 훈련 속도의 상당한 개선으로 이어짐

In [96]:
# 훈련 프로그램 실행에 집중하기 위해 requireOnDisk_bool 파라미터 사용
# 디스크 상에서 시리즈 UID 없는 데이터는 거를 예정

In [97]:
# 애노테이션 정보는 series_uid로 그룹화하여 일치하는 행 찾아내어 직경 정보 합치기
# 데이터의 xyz좌표가 반경의 절반이상 차이나면 직경 0.0으로 간주
# 훈련셋과 검증셋에 대해 좋은 결절 크기 분포 가지도록 필터링 할 것이므로 부정확한 직경값을 가져도 됨(나중에 문제 생겼을 때를 대비해 이러한 작업한 사실 기억)

In [98]:
# 결절의 직경 합치는 데 많은 코드 사용되었지만 이런 데이터 처리 및 매칭은 원본 데이터에 따라 의존적이나 매우 흔한 작업임

In [99]:
# 데이터 정렬 후 반환
# 결절 샘플을 크기 정보로 내림차순 정렬

In [100]:
# noduleInfo_list의 튜플 멤버 순서는 위의 정렬
# 일부 CT 단면들을 모아 결절 직경에 대해 잘 분포된 실제 결절을 반영하는 덩어리 얻어올 수 있게 됨

## 3절 - 개별 CT 스캔 로딩

In [1]:
# 디스크에서 CT 데이터 얻어와 파이썬 객체로 변환해 3차원 결절 밀도 데이터로 사용할 수 있도록 만드는 작업
# .mhd와 .raw 파일로부터 Ct 객체에 연결된 경로
# 이 맵을 활용하여 관심있는 부분 추출하기 위해 데이터를 주소로 접근 가능하게 해야 함

In [2]:
# CT 스캔 파일의 원래 포맷은 DICOM임
# 최초 버전은 1984년에 만들어졌고 그 시절 컴퓨팅 관련 내용들은 다소 더러움

In [3]:
# LUNA는 데이터를 MetaIO 포맷으로 변환해 놨고 사용하기 쉬움
# 들어본 적 없는 포맷이라도 블랙박스로 간주하고 넘파이 배열로 읽기 위해 SimpleIKT 사용

In [4]:
# 실제 프로젝트라면 원본 데이터에 어떤 정보 타입이 포함되는지 이해하고 싶겠지만 지금은 서드파티에 의존해도 됨
# 입력 데이터에 대한 모든 상세 내용 파악하는 것과 데이터 로딩 라이브러리가 넘겨주는 결과의 세부 내용을 모른 채 사용하는 것 사이에서 균형을 유지하는 데에는 경험 필요

In [5]:
# 주어진 데이터 샘플을 식별 가능하면 편리함
# 어떤 샘플이 문제 일으키는지 명확하게 잡아 논의할 수도 있고 분류가 안되는 샘플만 잡아서 이슈를 디버깅해 품질을 높일 수도 있음
# 샘플 특성에 따라 식별자가 숫자나 문자열 같은 단일 값일 수도 있고 튜플처럼 복잡한 형태 가질 수도 있음

In [6]:
# 특정 CT 스캔 식별을 위해 시리즈 UID 사용 중임
# DICOM은 개별 DICOM 파일, 여러 파일 그룹, 처리 과정 등에 단일 식별자(UID) 주로 사용
# 개념은 UUID 개념과 흡사하지만 생성하는 방식이나 포맷은 다름
# 여기서는 UID를 여러 CT 스캔 참조 시 사용할 ASCII 문자열로 만든 단일 키로 취급함

In [8]:
# 10개 서브셋에는 각각 90개, 전체 888개의 CT 스캔이 있으며, 각 CT스캔은 .mhd와 .raw 확장자 가지는 두 개의 파일로 나뉨
# 여러 파일로 나뉜 데이터의 세부는 sitk루틴에 가려져 있음
# ct_a는 3차원 배열이고 세 개의 차원은 공간을 나타내고 하나는 밀도를 나타냄
# 텐서에서 채널 정보는 네 번째 차원으로 표현되며 크기는 1

In [9]:
# __init__ 메소드 작성 시 ct_a 값을 지워줄 필요 있음
# CT 스캔 복셀은 하우스필드 단위(Housefield Unit)로 표시하는데 공기는 -1000HU(약 0g/cc), 물은 0HU(1g/cc), 뼈는 1000HU(2~3g/cc)임
# 어떤 CT 스캐너는 스캔 영역을 벗어난 복셀을 나타내기 위해 밀도에 음의 값 사용
# 환자의 몸 외부는 공기이므로 시야에 해당하는 영역을 위해 값이 -1000HU 이하면 버림
# 뼈나 금속 이식물 등도 필요 없으므로 2g/cc(1000HU) 이상도 버림
# 관심사인 종양의 경우 데체로 1g/cc(0HU) 근처이므로 1g/cc가 아닌 경우도 버림

In [10]:
# 위와 같이 데이터에서 이상값을 제거해야 함
# 목표와 거리가 먼 값들을 사용하면 배치 정규화를 위한 통계 값이 왜곡되어 최적으로 이뤄지지 않는 등 모델 훈련에 어려움을 겪게 됨

In [11]:
# 데이터 범위가 -1000~1000 사이인 점 인식
# 샘플에 정보 채널 넣을 때 HU와 부가적인 데이터 사이의 차이점을 고려하지 않으면 원본 HU값이 새 채널값 가려버릴 수 있음
# 프로젝트의 분류 단계에서는 채널을 더하지 않으므로 지금은 별도의 처리가 필요하지 않음

## 4절 - 환자 좌표계를 사용해 결절 위치 정하기

In [12]:
# 통상적으로 딥러닝 모델은 입력 뉴런 수가 고정되어 있어 고정된 크기의 입력을 필요로 함
# 분류기의 입력으로 사용할 고정된 크기의 결절 후보를 담을 배열 만들 수 있어야 함
# CT 스캔에서 깔끔하게 잘라낸 중심이 잘 잡힌 후보 데이터를 사용해서 모델이 입력 언저리에 감춰진 결절 탐지하는 일 없게 만들기
# 입력의 다양성을 줄여 상대적으로 모델이 수행할 작업 수월하게 만드는 것

In [14]:
# 앞에서 읽어들이 후보 중심점 데이터는 복셀이아닌 밀리미터 단위로 표시됨
# 밀리미터 기반 좌표계인 (X, Y, Z)로부터 CT 스캔 단면 데이터 배열에서 사용한 복셀 주소 기반 좌표계인 (I, R, C)로 좌표 변환

In [16]:
# 환자 좌표계에서 양의 X값은 환자의 왼쪽, 양의 Y값은 환자의 뒤쪽, 양의 Z 값은 환자의 머리 방향임
# 왼쪽-후면-상부(Left-Posterior-Superior)를 줄여 LPS라고도 함
# 밀리미터 단위로 측정하며, 위치 기준을 임의로 잡기 때문에 CT 복셀 배열의 기준과는 일치하지 않음
# 특정 스캔과는 무관하게 해부학적으로 관심있는 위치를 지정하기 위해 사용됨
# CT 배열과 환자 좌표계 사이의 관계를 정의하는 메타데이터는 DICOM 파일의 헤더에 저장되어 있고 메타 영상 형식으로 영역 지정되어 있음
# 이 메타데이터는 (X, Y, Z)에서 (I, R, C)로의 변환을 가능하게 함

In [18]:
# CT 스캔마다 복셀 크기가 다르고 정육면체가 아님
# 대부분 행과 열은 수치가 같고 인덱스는 수치가 조금 크고 다른 비율을 가지는 복셀 크기도 존재할 수 있음
# 정방형의 픽셀로 그리면 왜곡되게 보이므로 비율 계수 적용해야 함
# 이런 세부 사항 알면 결과를 시각적으로 해석할 때 도움됨

In [19]:
# CT는 일반적으로 512행, 512열로 구성되며 인덱스 차원은 대략 100~250개의 단면으로 구성
# 약 2^25개의 복셀에 해당하며 3200만 개의 데이터 포인트
# 각 CT는 파일 메타데이터 내에 복셀의 크기를 밀리미터 단위로 정의하며 이를 참조하기 위해 ct_mhd.GetSpacing() 호출

In [21]:
# 밀리미터 좌표와 (I, R, C) 배열 좌표 변환을 돕기 위한 유틸리티 코드 정의
# 축을 뒤집는 것(회전 등의 변환)은 ct_mhd.GetDirections()가 반환하는 튜플에 3*3 행렬로 인코딩 되어 있음
# 복셀 인덱스를 좌표로 바꾸기 위해
# 1. 좌표를 XYZ 체계로 만들기 위해 IRC에서 CRI로 뒤집기
# 2. 인덱스를 복셀 크기로 확대 축소
# 3. 파이썬의 @를 사용해 방향을 나타내는 행렬과 행렬곱 수행
# 4. 기준으로부터 오프셋 더하기
# XYZ에서 IRC변환은 각 단계 및 순서 역순
# 복셀의 크기는 네임드 튜플이 가지고 있으므로 이것들도 배열로 변경

In [22]:
# 환자 좌표를 배열 좌표로 변환할 때 필요한 메타데이터는 CT 데이터와 함꼐 들어 있는 MetaIO 파일에 포함되어 있음
# .mhd 파일에서 복셀 크기와 포지셔닝 메타데이터를 꺼내면 ct_a 얻을 수 있음

In [23]:
# 복셀의 99.9999%는 결절 부분 아니므로 각 후보 영역을 추출해 모델이 한 번에 한 영역에 집중할 수 있도록 만들 예정
# 모델이 해결할 문제의 범위를 줄일 방법을 찾는 것은 특히 프로젝트 초기에 첫 번째 구현부를 동작시키는데 도움됨

In [ ]:
# getRawNodule 함수는 LUNA CSV 데이터에 명시된 환자 좌표계로 표시된 중심 정보와 복셀 단위의 너비 정보도 인자로 전달받아 정육면체의 CT 덩어리와 배열 좌표로 변환된 후보의 중심값 반환

## 5절 - 간단한 데이터셋 구현

In [24]:
# Dataset을 서브클래싱하는 것은 임의의 데이터를 파이토치 생태계와 연결하는 것
# 각 Ct 인스턴스는 모델을 훈련시키거나 효과 검증할 때 사용하는 수백 개의 샘플
# LunaDataset 클래스는 이런 샘플을 정규화하고 각 CT의 결절은 평탄화 작업을 통해 어느 Ct 객체에서 가져온 샘플인지 상관없이 인출 가능하도록 하나의 단일 컬렉션으로 합쳐짐

In [25]:
# 구현은 Dataset 서브클래싱에 필요한 요구사항으로 시작해 거꾸로 가면서 작업
# 클래스를 직접 구현하고 인스턴스화하면 이전의 예제처럼 사용 가능
# 만들 커스텀 서브클래스는 파이토치API가 요구하는 두 함수만 구현하면 됨
# 1. 초기화 후에 하나의 상수값을 반환해야 하는 __len__구현
# 2. 인덱스를 인자로 받아 훈련(또는 검증)에서 사용할 샘플 데이터 튜플을 반환하는 __getitem__메소드

In [26]:
# __len__ 구현은 우리가 가진 후보 리스트 하나하나가 샘플이며 데이터셋은 우리가 가진 샘플 수만큼 크므로 구현 간단함
# 따라야 할 유일한 규칙은 __len__이 N값 반환하면 __getitem__은 0~N-1까지의 입력값에 유효한 아이템 넘기는 것

In [27]:
# __getitem__은 ndx 인자를 받아 4 개의 아이템 있는 샘플 튜플 반환
# self.cadidateInfo_list 구현하고 getRawNodule 함수 제공해야 함
# 이후 코드에서 사용할 데이터를 적절한 타입과 배열 차원으로 준비하는 역할
# 가능한 후보 클래스 요소를 가지는 분류 텐서 만들어야 함

In [28]:
# LunaDataset으로부터 쓸만한 성능을 얻으려면 온디스크 캐싱(on-disk caching)에 시간을 들일 필요 있음
# 모든 샘플에 대해 디스크로부터 전체 CT 스캔을 읽는 부담 피할 수 있음
# 캐시를 사용하지 않으면 50배는 더 느려질 것임
# 진행하는 프로젝트의 병목 지점이 어디인지 파악하고 최적화하는 데 주의를 기울여야 함

In [29]:
# Ct.getCtRawCandidate에 쓰인 파일 캐시 래퍼 함수 사용
# 이전과 다른 캐싱 메소드 확인
# getCt의 반환값을 메모리에 캐싱해 동일한 Ct 인스턴스에 대한 요청은 디스크에서 모든 데이터를 다시 읽을 필요 없게 대응
# 반복되는 요청에 대해서 엄청난 속도 향상 가져다 주지만 메모리에 하나의 CT만 있으므로 접근 순서 신경쓰지 않으면 잦은 캐시 미스 발생

In [31]:
# getCt 호출하는 getCtRawCandidate함수도 출력을 캐싱함
# 이 캐시된 값 사용할 경우 getCt는 호출되지 않음
# 이 값은 파이썬의 diskcache 라이브러리 사용하여 디크스에 캐시됨
# 동일한 데이터의 두 번째 전달부터는 입력값의 I/O 시간은 더 고려하지 않아도 됨

In [32]:
# 모든 프로젝트는 샘플을 훈련셋과 검증셋으로 나누는 과정 가짐
# val_stride 파라미터 사용해 샘플 중 10번째에 해당하는 모든 경우 검증셋으로 둠
# 그리고 isValSet_bool파라미터로 훈련 데이터나 검증 데이터만 둘지 둘 다 둘지를 지정함
# 코드는 series_uid에 값 넣으면 해당 데이터의 결절만 인스턴스에 담음 -> 문제 있는 CT 스캔하나만 자세히 볼 수 있어 데이터의 시각화나 디버깅에 유용함

In [33]:
# Dataset의 N번째 데이터만 모아 모델 검증용 서브셋 제작
# 서브셋 처리는 isValiSet_bool 인자값에 의존

In [34]:
# 이렇게 두 개의 Dataset 인스턴스 만들고 훈련 데이터와 검증 데이터 분리함
# self.candidateInfo_list의 정렬 순서가 고정되어 있다는 가정이 필요한 과정
# getCandidateInfoList 함수가 리스트 반환 전에 리스트를 정렬하며, 따라서 후보 정보를 가지고 있는 튜플이 고정된 순서를 가지는 것을 보장

In [37]:
# 주의사항으로 훈련 데이터와 검증 데이터의 분리 작업에서 어떤 환자 데이터는 훈련이나 테스트 샘플 중 한 곳에서만 보이게 만들어야 함
# 현재 이런 점은 문제되지 않지만 문제 되면 환자 리스트와 CT 스캔을 분리한 후 결절 레벨에 대한 작은 작업을 진행

In [38]:
# 분리된 두 데이터는 기대되는 모든 입력 유형 포함해야하고, 두 세트의 샘플은 모두 예상 가능한 수준의 입력값으로 구성되야 하고, 훈련셋은 실세계 데이터로 보기 어려운 값을 검증셋용 힌트로 가지면 안 됨

In [ ]:
# 데이터 렌더링을 이용해 입력이 어떤 형상인지에 대한 직관을 키울 수 있음
# 문제를 추적하고 조사할 때 이미지를 보고 해결 방법을 찾는 직관력을 키워줄 수도 있음

## 6절 - 결론

In [39]:
# 파이토치가 우리 데이터에 대해 감을 잡도록 만듦
# DICOM 메타 이미지 원본 데이터를 텐서로 변환했고 모델을 구현하고 훈련 루프 시작할 준비 완료함

## 7절 - 연습 문제

In [40]:
# 1. LunaDataset 인스턴스를 순회하는 프로그램 작성하고 시간을 재어보자. 처음에는 N=1000샘플로 제한하여 실행해야 할 것이다
#    a. 처음 실행할 때 걸린 시간은 얼마인가?
#    b. 두 번째 실행할 때 걸린 시간은 얼마인가?
#    c. 캐시를 클리어하면 실행 시간에 어떤 영향을 주는가?
#    d. lastN=1000 샘플을 사용하면 첫 번째/ 두 번째 시간에 어떤 영향을 주는가?

In [41]:
# 2. LunaDataset 구현을 바꿔서 __init__에서 샘플을 섞어보자(randomize). 캐시를 클리어하고 수정된 버전을 실행하라. 처음과 두 번째 실행 시간에 어떤 변화가 있는가?

In [42]:
# 3. 랜덤화를 취소하고 getCt에 있는 @functools.lru_cache(1, typed=True) 데코레이터를 주석 처리한 다음에 수정 버전을 실행하자. 수행 시간이 바뀌었는가?